In [ ]:
#!pip install matplotlib
#!pip install tqdm
#!pip install tensorflow
#!pip install keras

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.simplefilter('ignore')

# Семинар 6 - Погружение в глубокое обучение
В семинаре, будем использовать набор данных `fashion_mnist`, загрузим их

In [ ]:
from keras.datasets import fashion_mnist
(x_train, y_train_cat), (x_test, y_test_cat) = fashion_mnist.load_data()
print('Training data shape: ', x_train.shape)
print('Test data shape: ', x_test.shape)

num_classes = 10
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 
               'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

Сгенерируем случаные примеры для каждого класса и посмотрим на них.

In [ ]:
fig = plt.figure(figsize=(15,5))
for i in range(num_classes):
    ax = fig.add_subplot(2, 5, 1 + i, xticks=[], yticks=[])
    idx = np.where(y_train_cat[:]==i)[0]
    features_idx = x_train[idx,::]
    img_num = np.random.randint(features_idx.shape[0])
    im = features_idx[img_num]
    ax.set_title(class_names[i])
    plt.imshow(im, cmap='gray_r')
plt.show()

##  Построим нашу первую нейросеть
Импорт `Keras`

In [ ]:
from keras.models import Sequential # Модель, где все слои соединены друг с другом
from keras.layers import Dense, Flatten, Activation # Слой, где все нйероны предыдущего уровня соединены с нейронами следующего
from keras.utils import np_utils
from keras.optimizers import SGD,Adam,RMSprop

### Проведем небольшие предобработки

In [ ]:
y_train = np_utils.to_categorical(y_train_cat, num_classes=num_classes)
y_test = np_utils.to_categorical(y_test_cat, num_classes=num_classes)

### Построим модель

In [ ]:
input_shape = x_train.shape[1:]

In [ ]:
# Создаем последовательную модель
model = Sequential()

# Добавляем слои
model.add(Flatten(input_shape=input_shape))
model.add(Dense(200))
model.add(Activation('sigmoid'))
model.add(Dense(100))
model.add(Activation('sigmoid'))
model.add(Dense(10))
model.add(Activation('softmax'))

# Компилируем модель
optimizer = SGD(lr=0.1)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
print(model.summary())

#### Вопрос: 
Почему: $784 \times 200 = 156 800$, а сетка показывает число параметров: $157000$?

### Обучим модель, задав параметры

In [ ]:
batch_size = 1000 # Выбираем размер Батча
epochs = 100

In [ ]:
# Обучаем модель! 
history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

In [ ]:
plt.plot(history.history['acc'], label='train')
plt.plot(history.history['val_acc'], label='val')
plt.xlabel('Epoch')
plt.ylabel('acc')
plt.legend()
plt.show()

In [ ]:
def viz_history(history):
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='val')
    plt.xlabel('Epoch')
    plt.ylabel('error')
    plt.legend()
    plt.show()

viz_history(history)

## Что мы можем улучшить? 
- Отнормировать признаки
- Заменить сигмоиды на ReLu
- Задать правила инициации весов

### Нормирование
<img src='normalize.png'>

Отнормируйте и центрируйте признаки:

In [ ]:
#  Ваш Код здесь

### Функции активации
<img src='activations.png'>

### Инициациия весов
__Случайно__  
$ w = a * random$, но тогда если $a \gg 1$, то на выходе $b\gg1$ и если $a \ll 1 $, то $b \approx 0 $  

__Xavier__  
$a = \frac{1}{\sqrt{n}}$, где $n$ - кол-во нейронов на входе

__He__  
$a = \frac{1}{\sqrt{\frac{n}{2}}}$, где $n$ - кол-во нейронов на входе

In [ ]:
from keras.initializers import he_normal

В функции `create_model()` ниже: 
1. Добавьте инициацию весов в полносвязных слоях с помощью функции `he_normal()`
2. Измените функцию активации полносвязных слоев на Relu

In [ ]:
def create_model(input_shape, optimizer):
    model = Sequential()
    model.add(Flatten(input_shape=input_shape))
    model.add(Dense(200))
    model.add(Dense(100))
    model.add(Dense(10))
    model.add(Activation('softmax'))
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
optimizer = SGD(lr=0.1)
model = create_model(input_shape, optimizer)

In [ ]:
history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

In [ ]:
def viz_history(history):
    plt.plot(history.history['acc'], label='train')
    plt.plot(history.history['val_acc'], label='val')
    plt.xlabel('Epoch')
    plt.ylabel('acc')
    plt.legend()
    plt.show()

viz_history(history)

In [ ]:
def viz_history(history):
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='val')
    plt.xlabel('Epoch')
    plt.ylabel('error')
    plt.legend()
    plt.show()

viz_history(history)

## Влияние скорости обучения
Посмотрим, как влияет параметр `learning_rate` на качество нашей модели на обучающей выборке

Перебирите значения `[0.001, 0.01, 0.1, 1]` для `learning_rate` и постройте графики зависимости доли правильных ответов (или loss-функции) в зависимости от значения параметра:

In [ ]:
# Ваш код здесь

## Влияние метода оптимизации градиентного спуска

<img src='optimizers7.gif'>

#### Momentum
Вместо того, чтобы использовать только градиент текущего шага, мы будем накапливать импульс градиента прошлых шагов для определения направления движения. 
В связи со стохастической природой, обновления градиента происходят "зигзагообразно", с помощью момента мы усиливаем движение вдоль основного направления. На практике коэффициент у момента инициализируется на уровне 0,5 и постепенно увеличивается до 0,9 в течение нескольких эпох. 
  
#### Adagrad
Некоторые признаки могут быть крайне информативными, но встречаться редко. Экзотическая высокооплачиваемая профессия, причудливое слово в спам-базе — они запросто потонут в шуме всех остальных обновлений. Речь идёт не только о редко встречающихся входных параметрах. Скажем, вам вполне могут встретиться редкие графические узоры, которые и в признак-то превращаются только после прохождения через несколько слоёв свёрточной сети. Хорошо бы уметь обновлять параметры с оглядкой на то, насколько типичный признак они фиксируют. Достичь этого несложно: давайте будем хранить для каждого параметра сети сумму квадратов его обновлений. 
 
#### RMSProp (Root Mean Square Propogation)   
Мы обновляем меньше веса, которые слишком часто обновляются (идея Adagrad), при этом используем полную суммы обновлений, и используем усреднённый по истории квадрат градиента.

#### Adam (Adaptive moment estimation)
Cочетает в себе и идею накопления движения и идею более слабого обновления весов для типичных признаков

In [ ]:
names = ['SGD with Momentum', 'Adam', 'RMSprop']
optimizers = {'SGD with Momentum': SGD(nesterov=True), 
              'Adam': Adam(),
              'RMSprop': RMSprop()
             }

for opt in tqdm(names):
    model = create_model(input_shape, optimizers[opt])
    history = model.fit(x_train, y_train, verbose=0,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
    plt.plot(history.history['val_acc'], label='optimizers = {}'.format(opt))
    
plt.xlabel('Epoch')
plt.ylabel('Error')
plt.legend()
plt.show()

## Применим, все полученные знания

In [ ]:
optimizer = Adam()
model = create_model(input_shape, optimizer)

history = model.fit(x_train, y_train, verbose=1,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

In [ ]:
viz_history(history)

## Объявляем, борьбу с переобучением

In [ ]:
from keras.layers import Dropout, BatchNormalization

Добавьте в сеть слои `Dropout` или `BatchNormalization` после `Dense`

In [ ]:
# Ваш код здесь

In [ ]:
history = model.fit(x_train, y_train, verbose=1,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

In [ ]:
viz_history(history)

# Теперь твоя очередь! 

Побей бейзлайн 2 в соревновании [Птица или самолет](https://www.kaggle.com/c/bird-or-aircraft-dafe-open/overview)

# Ссылки
- [Курс "Deep learning на пальцах", лекция 4](https://youtu.be/tnrbx7V9RbA)
- [Статья: Оптимизация градиентного спуска](http://ruder.io/optimizing-gradient-descent/)
- [Статья: Методы оптимизации нейронных сетей](https://habr.com/ru/post/318970/)